# **An Omakase Resturant in London or the outer boroughs**

**Business Problem**

A group of young F&B entrepreneurs are looking for a new location to invest in an omakase restaurant serving fusion food. However, as London is a very expensive location for any budding resturanteur, and finding the right location to have your first restaurant could dictate how you balance your cash flow along with your human traffic flow. Too expensive a location – your rentals will cut too deeply into your operating budget. On the flipside, if the location is not suitable, without enough human traffic, there is no positive cash flow to sustain the business.

**Who would be interested?**

Budding restaurant entrepreneurs and potential café owners who want to open a shop in London but are put off by the steep rental prices and are looking for alternative locations where there are already existing amenities, adequate public transport and a potentially complimenting ecosystem of other F&B outlets.


**Import the necessary libraries**

In [ ]:
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors


url='https://en.wikipedia.org/wiki/List_of_London_boroughs'

LDF=pd.read_html(url, header=0)[0]

LDF.head()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E,25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


Renaming the columns, for ease of understanding. Moving ahead, due to the variing differences in rental prices, we have only specifcied the outer bouroughs that are within "£90-125"

In [ ]:
LF = LDF.drop(['Status','Local authority','Political control','Headquarters','Nr. in map'], axis=1)
LF['Inner'].replace(np.nan,'0', inplace=True)
LF['Borough'].replace('Barking and Dagenham [note 1]','Barking and Dagenham', inplace=True)
LF['Borough'].replace('Greenwich [note 2]','Greenwich', inplace=True)
LF['Borough'].replace('Hammersmith and Fulham [note 4]','Hammersmith and Fulham', inplace=True)
Inn = ['Camden','Greenwich','Hackney','Hammersmith and Fulham','Islington','Kensington and Chelsea','Lewisham','Lambeth','Southwark','Tower Hamlets','Wandsworth','Westminster']
LF.head()
LF['Inner'] = '0'
LF.head()

,Borough,Inner,Area (sq mi),Population (2013 est)[1],Co-ordinates
0,Barking and Dagenham,0,13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E
1,Barnet,0,33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W
2,Bexley,0,23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E
3,Brent,0,16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W
4,Bromley,0,57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E


In [ ]:

LF['Inner'] = LF.Borough.isin(Inn).astype(int)
Out = LF[LF.Inner == 0]
Out = Out.drop(['Inner'], axis=1)
df = Out.rename(columns = {"Area (sq mi)": "Area", 
                            "Population (2013 est)[1]":"Population"})
geolocator = Nominatim(user_agent="London_explorer")
df['Co-ordinates']= df['Borough'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df[['Latitude', 'Longitude']] = df['Co-ordinates'].apply(pd.Series)
df.head()

,Borough,Area,Population,Co-ordinates,Latitude,Longitude
0,Barking and Dagenham,13.93,194352,"(51.5541171, 0.15050434261994267)",51.554117,0.150504
1,Barnet,33.49,369088,"(51.65309, -0.2002261)",51.653090,-0.200226
2,Bexley,23.38,236687,"(39.9692378, -82.936864)",39.969238,-82.936864
3,Brent,16.70,317264,"(32.9373463, -87.1647184)",32.937346,-87.164718
4,Bromley,57.97,317899,"(51.4028046, 0.0148142)",51.402805,0.014814


In [ ]:

Max_Rent = ['102.25','150.75','97','150.75','118.5','129.25','140','102.25','107.75','140','86','161.5','161.5','140','123.75','134.5','118.5','140','129.25','145.25']
df['Max_Rent'] = Max_Rent

df["Max_Rent"] = pd.to_numeric(df["Max_Rent"])
fin = df[df.Max_Rent <= 125]
fin

Long_list = fin['Longitude'].tolist()
Lat_list = fin['Latitude'].tolist()
print ("Old latitude list: ", Lat_list)
print ("Old Longitude list: ", Long_list)

replace_longitudes = {-106.6621329:0.0799, -2.8417544: 0.1837}
replace_latitudes = {50.7164496:51.6636, 51.0358628: 51.5499}

longtitudes_new = [replace_longitudes.get(n7,n7) for n7 in Long_list]
latitudes_new = [replace_latitudes.get(n7,n7) for n7 in Lat_list]

fin = fin.drop(['Co-ordinates', 'Longitude'], axis=1)

fin['Longitude'] = longtitudes_new
fin['Latitude'] = latitudes_new
fin

Old latitude list:  [51.5541171, 39.9692378, 51.4028046, 51.6520851, 51.587929849999995, 51.0043613, 51.410802849999996, 51.5763203]
Old Longitude list:  [0.15050434261994267, -82.936864, 0.0148142, -0.0810175, -0.10541010599099046, -2.337474942629507, -0.18809850595572736, 0.0454097]


,Borough,Area,Population,Latitude,Max_Rent,Longitude
0,Barking and Dagenham,13.93,194352,51.554117,102.25,0.150504
2,Bexley,23.38,236687,39.969238,97.00,-82.936864
4,Bromley,57.97,317899,51.402805,118.50,0.014814
8,Enfield,31.74,320524,51.652085,102.25,-0.081018
12,Haringey,11.42,263386,51.587930,107.75,-0.105410
14,Havering,43.35,242080,51.004361,86.00,-2.337475
22,Merton,14.52,203223,51.410803,123.75,-0.188099
24,Redbridge,21.78,288272,51.576320,118.50,0.045410


In [ ]:

address = 'London'

geolocator = Nominatim(user_agent="London_explorer")
location = geolocator.geocode(address)
London_latitude = location.latitude
London_longitude = location.longitude
print('The geograpical coordinates of London are {}, {}.'.format(London_latitude, London_longitude))

The geograpical coordinates of London are 51.5073219, -0.1276474.


**Using Folium to make a map**

In [ ]:
!pip install folium
import folium

Fin_Brgh = folium.Map(location=[London_latitude, London_longitude], zoom_start=12)


for lat, lng, label in zip(fin['Latitude'], fin['Longitude'], 
                            fin['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#Blue',
        fill_opacity=0.7).add_to(Fin_Brgh)
Fin_Brgh

**Entering my FourSquare ID**

In [ ]:

CLIENT_ID = 'PYTGNMOMS32H0D5EETGAPUX4OELK1NXL35WLDGBTOEPGJR2B' #'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'ECHKWOEU52OUZHETEWXUW05LKXDQ0KGUSSU33EHKKM0OWD5S' #'your-client-secret' # your Foursquare Secret
VERSION = '20190915' # Foursquare API version

print('My credentails:')
print('My CLIENT_ID: ' + CLIENT_ID)
print('My CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
My CLIENT_ID: PYTGNMOMS32H0D5EETGAPUX4OELK1NXL35WLDGBTOEPGJR2B
My CLIENT_SECRET:ECHKWOEU52OUZHETEWXUW05LKXDQ0KGUSSU33EHKKM0OWD5S


In [ ]:
radius = 5000
LIMIT = 100

def getVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue_Lat', 
                  'Venue_Long', 
                  'Venue_Category']
    
    return(nearby_venues)

In [ ]:

Brgh_Venues = getVenues(names=fin['Borough'],
                        latitudes=fin['Latitude'],
                        longitudes=fin['Longitude'])

Barking and Dagenham
Bexley
Bromley
Enfield
Haringey
Havering
Merton
Redbridge


**Proceeding to count the venue to get the data scale**

In [ ]:
Brgh_Venues.groupby('Borough').count()

,Latitude,Longitude,Venue,Venue_Lat,Venue_Long,Venue_Category
Borough,,,,,,
Barking and Dagenham,97,97,97,97,97,97
Bexley,88,88,88,88,88,88
Bromley,100,100,100,100,100,100
Enfield,100,100,100,100,100,100
Haringey,100,100,100,100,100,100
Havering,4,4,4,4,4,4
Merton,100,100,100,100,100,100
Redbridge,100,100,100,100,100,100


**Sorting the data frame with the most popular venues.**

In [ ]:
London_Brgh_onehot = pd.get_dummies(Brgh_Venues[['Venue_Category']], prefix="", prefix_sep="")
mid =  Brgh_Venues['Borough']

London_Brgh_onehot.insert(0, 'Borough', mid)

London_Brgh_onehot.head()

,Borough,ATM,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Business Service,Café,Caribbean Restaurant,Cave,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,...,Portuguese Restaurant,Print Shop,Pub,Recreation Center,Rental Car Location,Reservoir,Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Soccer Stadium,Spanish Restaurant,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tapas Restaurant,Taxi Stand,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
Brgh_grouped = London_Brgh_onehot.groupby('Borough').mean().reset_index()
Brgh_grouped

,Borough,ATM,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Business Service,Café,Caribbean Restaurant,Cave,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,...,Portuguese Restaurant,Print Shop,Pub,Recreation Center,Rental Car Location,Reservoir,Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Soccer Stadium,Spanish Restaurant,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tapas Restaurant,Taxi Stand,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Barking and Dagenham,0.000000,0.010309,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.020619,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.010309,0.000000,0.010309,0.00,0.000000,0.00,0.020619,0.000000,0.041237,0.00,0.00,0.000000,0.00,0.020619,0.00,0.072165,0.000000,0.000000,0.000000,0.00,0.00,0.020619,...,0.020619,0.000000,0.072165,0.00,0.000000,0.00,0.000000,0.010309,0.00,0.000000,0.000000,0.010309,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.092784,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.010309,0.010309,0.00,0.010309,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000
1,Bexley,0.011364,0.011364,0.00,0.00,0.011364,0.00,0.00,0.011364,0.011364,0.011364,0.011364,0.011364,0.00,0.011364,0.00,0.011364,0.00,0.000000,0.011364,0.000000,0.00,0.022727,0.00,0.000000,0.011364,0.011364,0.00,0.00,0.034091,0.00,0.000000,0.00,0.045455,0.011364,0.011364,0.022727,0.00,0.00,0.000000,...,0.000000,0.011364,0.000000,0.00,0.011364,0.00,0.034091,0.034091,0.00,0.011364,0.011364,0.000000,0.00,0.011364,0.00,0.00,0.00,0.00,0.011364,0.011364,0.011364,0.011364,0.011364,0.00,0.011364,0.011364,0.00,0.00,0.000000,0.011364,0.00,0.000000,0.011364,0.00,0.022727,0.011364,0.00,0.00,0.011364,0.011364
2,Bromley,0.000000,0.010000,0.00,0.00,0.000000,0.01,0.01,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.00,0.010000,0.000000,0.000000,0.00,0.000000,0.01,0.000000,0.000000,0.040000,0.00,0.01,0.000000,0.00,0.020000,0.01,0.090000,0.000000,0.000000,0.000000,0.00,0.01,0.010000,...,0.020000,0.000000,0.080000,0.00,0.000000,0.00,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.010000,0.000000,0.050000,0.000000,0.000000,0.01,0.000000,0.000000,0.00,0.00,0.010000,0.000000,0.00,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000
3,Enfield,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.01,0.000000,0.000000,0.030000,0.000000,0.020000,0.00,0.000000,0.00,0.000000,0.02,0.000000,0.000000,0.000000,0.01,0.000000,0.00,0.000000,0.000000,0.050000,0.00,0.00,0.000000,0.00,0.010000,0.00,0.090000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,...,0.010000,0.000000,0.070000,0.01,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.01,0.00,0.00,0.00,0.000000,0.000000,0.050000,0.000000,0.000000,0.00,0.000000,0.020000,0.00,0.00,0.000000,0.010000,0.00,0.010000,0.070000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000
4,Haringey,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.040000,0.000000,0.020000,0.00,0.000000,0.00,0.000000,0.01,0.000000,0.000000,0.000000,0.01,0.020000,0.01,0.000000,0.000000,0.110000,0.01,0.00,0.000000,0.02,0.000000,0.01,0.040000,0.000000,0.010000,0.000000,0.00,0.00,0.000000,...,0.000000,0.000000,0.110000,0.00,0.000000,0.01,0.010000,0.010000,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.00,0.01,0.01,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.00

In [ ]:
num_top_venues = 5

for brgh in Brgh_grouped['Borough']:
    print("_________"+brgh+"________")
    temp = Brgh_grouped[Brgh_grouped['Borough'] == brgh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

_________Barking and Dagenham________
           venue  freq
0           Park  0.09
1    Supermarket  0.09
2  Grocery Store  0.08
3    Coffee Shop  0.07
4            Pub  0.07


_________Bexley________
                venue  freq
0         Pizza Place  0.06
1      Ice Cream Shop  0.05
2         Coffee Shop  0.05
3  Chinese Restaurant  0.03
4      Discount Store  0.03


_________Bromley________
           venue  freq
0    Coffee Shop  0.09
1            Pub  0.08
2  Grocery Store  0.08
3           Park  0.07
4    Supermarket  0.05


_________Enfield________
                venue  freq
0         Coffee Shop  0.09
1  Turkish Restaurant  0.07
2                 Pub  0.07
3       Garden Center  0.05
4         Supermarket  0.05


_________Haringey________
                venue  freq
0                Café  0.11
1                 Pub  0.11
2  Turkish Restaurant  0.08
3                Park  0.08
4         Coffee Shop  0.04


_________Havering________
               venue  freq
0        IT Service

In [ ]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
brgh_venues_sorted = pd.DataFrame(columns=columns)
brgh_venues_sorted['Borough'] = Brgh_grouped['Borough']

for ind in np.arange(Brgh_grouped.shape[0]):
    brgh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brgh_grouped.iloc[ind, :], num_top_venues)

brgh_venues_sorted.head(8)

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,Park,Supermarket,Grocery Store,Coffee Shop,Pub
1,Bexley,Pizza Place,Ice Cream Shop,Coffee Shop,Discount Store,Chinese Restaurant
2,Bromley,Coffee Shop,Grocery Store,Pub,Park,Supermarket
3,Enfield,Coffee Shop,Turkish Restaurant,Pub,Park,Garden Center
4,Haringey,Pub,Café,Park,Turkish Restaurant,Bakery
5,Havering,IT Services,Indoor Play Area,Electronics Store,Food & Drink Shop,Fish & Chips Shop
6,Merton,Pub,Park,Coffee Shop,Café,Bar
7,Redbridge,Pub,Park,Coffee Shop,Italian Restaurant,Café


In [ ]:
kclusters = 3

brgh_grouped_clustering = Brgh_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brgh_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 2, 1, 2, 2], dtype=int32)

In [ ]:
brgh_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)
Borough_merged = pd.merge(fin,brgh_venues_sorted, on='Borough')
Borough_merged

,Borough,Area,Population,Latitude,Max_Rent,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,13.93,194352,51.554117,102.25,0.150504,0,Park,Supermarket,Grocery Store,Coffee Shop,Pub
1,Bexley,23.38,236687,39.969238,97.00,-82.936864,0,Pizza Place,Ice Cream Shop,Coffee Shop,Discount Store,Chinese Restaurant
2,Bromley,57.97,317899,51.402805,118.50,0.014814,0,Coffee Shop,Grocery Store,Pub,Park,Supermarket
3,Enfield,31.74,320524,51.652085,102.25,-0.081018,0,Coffee Shop,Turkish Restaurant,Pub,Park,Garden Center
4,Haringey,11.42,263386,51.587930,107.75,-0.105410,2,Pub,Café,Park,Turkish Restaurant,Bakery
5,Havering,43.35,242080,51.004361,86.00,-2.337475,1,IT Services,Indoor Play Area,Electronics Store,Food & Drink Shop,Fish & Chips Shop
6,Merton,14.52,203223,51.410803,123.75,-0.188099,2,Pub,Park,Coffee Shop,Café,Bar
7,Redbridge,21.78,288272,51.576320,118.50,0.045410,2,Pub,Park,Coffee Shop,Italian Restaurant,Café


**Limiting the number of clusters to 3**

In [ ]:

map_clusters = folium.Map(location=[London_latitude, London_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, rent, pop in zip(Borough_merged['Latitude'],
                                  Borough_merged['Longitude'],
                                  Borough_merged['Borough'],
                                  Borough_merged['Cluster Label'],
                                  Borough_merged['Max_Rent'],
                                  Borough_merged['Population']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + " " + "Rent " + str(rent) + " " + "Population " + str(pop), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=25,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:

Borough_merged.loc[Borough_merged['Cluster Label'] == 0, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]

,Borough,Area,Population,Max_Rent,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,13.93,194352,102.25,0,Park,Supermarket,Grocery Store,Coffee Shop,Pub
1,Bexley,23.38,236687,97.00,0,Pizza Place,Ice Cream Shop,Coffee Shop,Discount Store,Chinese Restaurant
2,Bromley,57.97,317899,118.50,0,Coffee Shop,Grocery Store,Pub,Park,Supermarket
3,Enfield,31.74,320524,102.25,0,Coffee Shop,Turkish Restaurant,Pub,Park,Garden Center


*What salient points emerge from this cluster are that a coffee shops are very popular in these locations. And to the author – data analyst, this is a good sign. As it indicates a potential synergistic relationship with other shops in the area that can act as a pull for human traffic.*

In [ ]:

Borough_merged.loc[Borough_merged['Cluster Label'] == 1, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]

,Borough,Area,Population,Max_Rent,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Havering,43.35,242080,86.0,1,IT Services,Indoor Play Area,Electronics Store,Food & Drink Shop,Fish & Chips Shop


*In the 2nd cluster, it seems that Havering is more of an electronics retail and IT Services location. While there are food and drink shops, the author – data analyst thinks that they are often more busy during office hours and will not have much human traffic potential after office hours, when it is often the most profitable period of F&B. However, an entrepreneur can consider this place, if the rental is low enough, but his business ought to be mature and already have a loyal customer base.*

In [ ]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 2, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]

,Borough,Area,Population,Max_Rent,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Haringey,11.42,263386,107.75,2,Pub,Café,Park,Turkish Restaurant,Bakery
6,Merton,14.52,203223,123.75,2,Pub,Park,Coffee Shop,Café,Bar
7,Redbridge,21.78,288272,118.50,2,Pub,Park,Coffee Shop,Italian Restaurant,Café


*In this third cluster, this location seems to have a right mix of pubs and restaurants, typical of an establish F&B scene. While this might seem ideal as there is already a ready-made human traffic, such a mature location might require higher rents, not something an entrepreneur who’s starting out might want. Also potentially, the pubs in the area could attract a rowdy crowd that might be at odds at the fine dining restaurant that we are looking to establish.*

**Summary of the results observed**
1.	Popular social – F&B venues outside of London are Coffee shops and Pubs
2.	The mix of Pubs and F&B places are well mixed and are not concentrated in specific clusters. 

**Discussion**
Looking at the data, I have been able to gleam that Bexley, Bromley and Enfield are suitable locations outside of London worth considering to open a restaurant. However, as the results are obtained by meshing different data sources, and one of the sources is Foursquare : -
1.	The higher a single ethnic group within a borough can significantly influence the popularity of that ethnic group’s cuisine.
2.	Only the larger stores might be available in Foursquare, smaller store might be spread by word of mouth.
3.	If the demographic and education level of the borough is either older or less educated or poor, then the access to such mobile apps will be limited, thus limiting the digital exposure in Foursquare. 

**Conclusion**
From the data available, the author strongly suggest that this Omakase restaurant be located in Enfield due to the desired ecosystem of coffee shops and other restaurants. Also there are less pubs in the area compared to other clusters which make the possibility of rowdy crowds lessen.
